라이브러리 불러오기

In [257]:
import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr

# 종목코드 or 종목명을 입력 받기

In [425]:
cate = input('종목명을 입력하시려면 1번, 종목코드를 입력하시려면 2번을 입력하세요 : ')
stock_name_list = fdr.StockListing('KRX')['Name'].to_list()
stock_num_list = fdr.StockListing('KRX')['Symbol'].to_list()

if cate == '1' :
    stock_name = input('종목명을 입력하세요 : ')
    if stock_name in stock_name_list :
        stock_num = stock_num_list[stock_name_list.index(stock_name)]
    else :
        print('존재하지 않는 종목명입니다.')
else :
    stock_num = input('종목코드을 입력하세요 : ')
    if stock_num in stock_num_list :
        stock_num = stock_num
    else :
        print('존재하지 않는 종목코드입니다.')


종목명을 입력하시려면 1번, 종목코드를 입력하시려면 2번을 입력하세요 : 1
종목명을 입력하세요 : ㄴ
존재하지 않는 종목명입니다.


# 종목코드 or 종목명으로 해당 종목의 주가 Data 불러오기

In [309]:
url = f'https://finance.naver.com/item/main.nhn?code={stock_num}'
tables = pd.read_html(url, encoding = 'cp949')

In [310]:
# 전일가격, 시가, 고가, 저가, 거래량, 거래대금 가져오기
price = pd.DataFrame(np.array([['전일', '시가', '고가', '저가', '거래량', '거래대금(단위 : 백만)'], 
          [int(tables[0].iloc[0, 0].split(' ')[2].replace(',', '')),
           int(tables[0].iloc[1, 0].split(' ')[2][-int(len(tables[0].iloc[1, 0].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[0, 1].split(' ')[2][-int(len(tables[0].iloc[0, 1].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[1, 1].split(' ')[2][-int(len(tables[0].iloc[1, 1].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[0, 2].split(' ')[2].replace(',', '')),
           int(tables[0].iloc[1, 2].split(' ')[2].replace(',', ''))
          ]]))

In [311]:
# 최근분기 재무정보 가져오기
re_info = tables[3].iloc[:, [0, -2]].T.reset_index(drop = True)

In [312]:
# 시가정보 가져오기
price_info = tables[5].T

In [313]:
# 외국인 정보 가져오기
fore = tables[6].T

In [314]:
# 동일 업종 비교 가져오기
si_cate = tables[9].T

In [315]:
# 데이터 합치기
stock_info = pd.concat([price, re_info, price_info, fore, si_cate], axis = 1)

# 해당 종목의 주가 Data 저장하기

In [316]:
# 오늘 날짜의 파일 이름 생성
file_name = datetime.datetime.today().strftime('%Y_%m_%d') + '_' + stock_num + '.csv'

In [321]:
# csv 파일로 저장
stock_info.to_csv(file_name, index = False)

In [322]:
# 잘 저장 되었는지 확인
pd.read_csv(file_name)

,0,1,2,3,4,5,0.1,1.1,2.1,3.1,...,15,0.2,1.2,2.2,3.2,0.3,1.3,2.3,0.4,1.4
0,전일,시가,고가,저가,거래량,거래대금(단위 : 백만),매출액,영업이익,당기순이익,영업이익률,...,배당성향(%),시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),동일업종 PER,동일업종 등락률
1,81900,81900,82100,80800,15013959,1218635,653885.0,93829.0,71417.0,14.35,...,NaN,"483조 5,524 억원",코스피 1위,5969782550,100원 l 1주,5969782550,3195921890,53.53%,17.24배,-0.94%


# 함수화 하기

In [423]:
def get_stock_info() :
    # 종목명 or 종목코드로 종목코드 확인
    cate = input('종목명을 입력하시려면 1번, 종목코드를 입력하시려면 2번을 입력하세요 : ')
    stock_name_list = fdr.StockListing('KRX')['Name'].to_list()
    stock_num_list = fdr.StockListing('KRX')['Symbol'].to_list()

    if cate == '1' :
        stock_name = input('종목명을 입력하세요 : ')
        if stock_name in stock_name_list :
            stock_num = stock_num_list[stock_name_list.index(stock_name)]
        else :
            return print('존재하지 않는 종목명입니다.')
    else :
        stock_num = input('종목코드을 입력하세요 : ')
        if stock_num in stock_num_list :
            stock_num = stock_num
        else :
            return print('존재하지 않는 종목코드입니다.')

    # 해당 종목의 데이터 가져오기
    url = f'https://finance.naver.com/item/main.nhn?code={stock_num}'
    tables = pd.read_html(url, encoding = 'cp949')
    
    # 전일가격, 시가, 고가, 저가, 거래량, 거래대금 가져오기
    price = pd.DataFrame(np.array([['전일', '시가', '고가', '저가', '거래량', '거래대금(단위 : 백만)'], 
          [int(tables[0].iloc[0, 0].split(' ')[2].replace(',', '')),
           int(tables[0].iloc[1, 0].split(' ')[2][-int(len(tables[0].iloc[1, 0].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[0, 1].split(' ')[2][-int(len(tables[0].iloc[0, 1].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[1, 1].split(' ')[2][-int(len(tables[0].iloc[1, 1].split(' ')[2]) / 2):].replace(',', '')),
           int(tables[0].iloc[0, 2].split(' ')[2].replace(',', '')),
           int(tables[0].iloc[1, 2].split(' ')[2].replace(',', ''))
          ]]))
    
    # 최근분기 재무정보 가져오기
    re_info = tables[3].iloc[:, [0, -2]].T.reset_index(drop = True)
    
    # 시가정보 가져오기
    price_info = tables[5].T
    
    # 외국인 정보 가져오기
    fore = tables[6].T
    
    # 동일 업종 비교 가져오기
    si_cate = tables[9].T
    
    # 데이터 합치기
    stock_info = pd.concat([price, re_info, price_info, fore, si_cate], axis = 1)
    return stock_info

In [424]:
get_stock_info()

종목명을 입력하시려면 1번, 종목코드를 입력하시려면 2번을 입력하세요 : 1
종목명을 입력하세요 : 삼전
존재하지 않는 종목명입니다.
